https://z-uo.medium.com/lenguage-translations-with-python-5dd745d1067f

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import time
import os
import pandas as pd

/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data=pd.read_csv('/Volumes/T7/mt-hemingway/data/book_google.csv', encoding='utf-8-sig')

In [3]:
data.columns

Index(['original', 'small', 'big', 'OPUS_small', 'OPUS_big', 'kakao_org',
       'M2M100_org', 'M2M100_small', 'kakao_small', 'M2M100_big', 'kakao_big',
       'google_org', 'google_small'],
      dtype='object')

In [4]:
data.shape

(768, 13)

In [24]:
it_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
it_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
# it_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
# it_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

In [30]:
translated_small=[]
for idx, val in tqdm(data.iterrows()):
    for_translate=val['small']
    try:
        encoded_input=it_en_tokenizer(for_translate, return_tensors='pt')
        output=it_en_model.generate(**encoded_input)
        out_text=it_en_tokenizer.batch_decode(output, skip_specual_tokens=True)
        out_text=out_text[0]
        out_text=out_text.replace('<pad>', '')
        time.sleep(0.5)
        translated_small.append(out_text)
    except TypeError:
        translated_small.append(val)
        time.sleep(0.5)
        continue

768it [30:04,  2.35s/it]


In [31]:
translated_big=[]
for idx, val in tqdm(data.iterrows()):
    for_translate=val['big']
    try:
        encoded_input=it_en_tokenizer(for_translate, return_tensors='pt')
        output=it_en_model.generate(**encoded_input)
        out_text=it_en_tokenizer.batch_decode(output, skip_specual_tokens=True)
        out_text=out_text[0]
        out_text=out_text.replace('<pad>', '')
        time.sleep(0.5)
        translated_big.append(out_text)
    except TypeError:
        translated_big.append(val)
        time.sleep(0.5)
        continue

768it [31:16,  2.44s/it]


In [24]:
data['OPUS_small']=translated_small
data['OPUS_big']=translated_big

In [36]:
data.to_csv('/Volumes/T7/mt-hemingway/data/book_OPUS.csv', index=False)

Kakao K2E

In [5]:
from kakaotrans import Translator
translator=Translator()
from tqdm import tqdm
import time

In [6]:
os.getcwd()

'/Users/mstudio/repository/hemingway'

In [7]:
translated_org=[]
for idx, val in tqdm(data.iterrows()):
    for_translate=val['original']
    try:
        trans=translator.translate(for_translate, src='en', tgt='kr')
        translated_org.append(trans)
        time.sleep(0.5)
    except TypeError:
        translated_org.append(trans)
        time.sleep(0.5)
        continue

768it [24:32,  1.92s/it]


In [12]:
data['kakao_org']=translated_org

In [16]:
data.to_csv('/Volumes/T7/mt-hemingway/data/book_kakao1.csv', index=False, encoding='utf-8-sig')

In [10]:
len(translated_small)

368

In [6]:
translated_small=[]
# data2=data[368:]
for idx, val in tqdm(data.iterrows()):
    for_translate=val['small']
    try:
        trans=translator.translate(for_translate, src='kr', tgt='en')
        translated_small.append(trans)
        time.sleep(0.5)
    except TypeError:
        translated_small.append(trans)
        time.sleep(0.5)
        continue

768it [24:16,  1.90s/it]


In [8]:
data['kakao_small']=translated_small

In [21]:
translated_big=[]
for idx, val in tqdm(data.iterrows()):
    for_translate=val['big']
    try:
        trans=translator.translate(for_translate, src='kr', tgt='en')
        translated_big.append(trans)
        time.sleep(0.5)
    except TypeError:
        translated_big.append(trans)
        time.sleep(0.5)
        continue

768it [24:19,  1.90s/it]


In [23]:
data['kakao_big']=translated_big

In [60]:
data.to_csv('data/kakao_book.csv', index=False, encoding='utf-8-sig')

In [36]:
data.to_csv('data/book5.csv', index=False, encoding='utf-8-sig')

In [30]:
data.columns

Index(['Unnamed: 0', 'original', 'small', 'big', 'OPUS_small', 'OPUS_big',
       'kakao_org', 'M2M100_org', 'M2M100_small', 'M2M100_big', 'kakao_small'],
      dtype='object')

M2M100

In [12]:
import torch
# Both should return 'True'
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [13]:
torch.has_mps

True

In [14]:
device=torch.device("mps")

In [11]:
from transformers import M2M100Config, M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="ko")

In [22]:
translated=[]
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="ko")
for i in tqdm(data['original']):
    try:
        encoded_hi=tokenizer(i, return_tensors='pt')
        generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("ko"))
        translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
    except TypeError:
        translated.append(i)
        time.sleep(0.5)
        continue

  0%|          | 0/768 [00:00<?, ?it/s]/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 768/768 [1:17:55<00:00,  6.09s/it]


In [23]:
data['M2M100_org']=translated

In [26]:
translated=[]
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="ko", tgt_lang="en")
for i in tqdm(data['small']):
    try:
        encoded_hi=tokenizer(i, return_tensors='pt')
        generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
        translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
    except TypeError:
        translated.append(i)
        time.sleep(0.5)
        continue

  0%|          | 0/768 [00:00<?, ?it/s]/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 768/768 [1:20:40<00:00,  6.30s/it]


In [28]:
data['M2M100_small']=translated

In [15]:
translated=[]
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="ko", tgt_lang="en")
for i in tqdm(data['big']):
    try:
        encoded_hi=tokenizer(i, return_tensors='pt')
        generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
        translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
    except TypeError:
        translated.append(i)
        time.sleep(0.5)
        continue

  0%|          | 0/768 [00:00<?, ?it/s]/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 768/768 [1:32:03<00:00,  7.19s/it]


In [19]:
data['M2M100_big']=translated

In [24]:
data.to_csv('/Volumes/T7/mt-hemingway/data/book_M2M100.csv', index=False, encoding='utf-8-sig')

Pororo https://github.com/kakaobrain/pororo

In [71]:
from pororo import Pororo

ImportError: cannot import name 'Pororo' from 'pororo' (unknown location)

In [65]:
from google_trans_new import google_translator
translator=google_translator()
translate_text=translator.translate('안녕하세요', lang_tgt='en')

/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'translate.google.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


google_new_transError: 404 (Not Found) from TTS API. Probable cause: Unknown

In [63]:
translator=Translator()
translator.detect('안녕하세요')

AttributeError: 'NoneType' object has no attribute 'group'

Papago

In [18]:
os.getcwd()

'/Users/mstudio/repository/hemingway'

In [1]:
from papago import Translator

ImportError: cannot import name 'Translator' from 'papago' (unknown location)

In [88]:
translated_org=[]
translator=Translator()
for idx, val in tqdm(data.iterrows()):
    for_translate=val['original']
    # try:
    trans=translator.translate(for_translate, source='en', target='ko')
    translated_org.append(trans)
    time.sleep(0.5)
    # except TypeError:
        # translated_org.append(trans)
        # time.sleep(0.5)
        # continue

0it [00:00, ?it/s]


KeyError: 'translatedText'

In [82]:
data.shape

(768, 11)

In [3]:
from papago import Translator
CLIENT_ID = 'MIbdxUCQUCIyzceuSFT7'  
CLIENT_SECRET = 'bBcUr1LOpZ'   
translator = Translator(CLIENT_ID, CLIENT_SECRET)
response = translator.translate('안녕하세요 아니 시벌 이게 뭔', 'ko', 'en')

ImportError: cannot import name 'Translator' from 'papago' (unknown location)

In [8]:
print(response)

Response(code=0, message=None, text=Hello. What is this?, source=ko)


In [7]:
import unicodedata
unicodedata.normalize('NFKD', print(response)).encode('ascii', 'ignore')

Response(code=0, message=None, text=Hello. I mean, what is this?, source=ko)


TypeError: normalize() argument 2 must be str, not None

In [ ]:
translated_org=[]
translator=Translator()
for idx, val in tqdm(data.iterrows()):
    for_translate=val['original']
    # try:
    trans=translator.translate(for_translate, source='en', target='ko')
    translated_org.append(trans)
    time.sleep(0.5)

In [118]:
print(test)

None


Googletrans

In [5]:
import googletrans

In [6]:
translator=googletrans.Translator()
translator.translate('mother fucker', src='en', dest='ko')
print(translator.translate('안녕하세요', dest='en').text)

hello


In [33]:
len(translated_org)

768

In [31]:
translated_org=translated_org+translated_org2

In [29]:
data2=data[730:]
translated_org2=[]
translator=googletrans.Translator()
for idx, val in tqdm(data2.iterrows()):
    for_translate=val['original']
    try:
        trans=translator.translate(for_translate, src='en', dest='ko')
        translated_org2.append(trans.text)
        time.sleep(0.5)
    except TypeError:
        translated_org2.append(trans.text)
        time.sleep(0.5)
        continue

38it [01:15,  1.97s/it]


In [34]:
data['google_org']=translated_org

In [10]:
data.to_csv('/Volumes/T7/mt-hemingway/data/book_google.csv', index=False, encoding='utf-8-sig')

In [1]:
translated_small

NameError: name 'translated_small' is not defined

In [7]:
translated_small=[]
translator=googletrans.Translator()
for idx, val in tqdm(data.iterrows()):
    for_translate=val['small']
    try:
        trans=translator.translate(for_translate, src='ko', dest='en')
        translated_small.append(trans.text)
        time.sleep(0.5)
    except TypeError:
        translated_small.append(trans.text)
        time.sleep(0.5)
        continue

768it [12:37,  1.01it/s]


In [9]:
data2=data[581:]
translated_small2=[]
translator=googletrans.Translator()
for idx, val in tqdm(data2.iterrows()):
    for_translate=val['small']
    try:
        trans=translator.translate(for_translate, src='ko', dest='en')
        translated_small2.append(trans.text)
        time.sleep(0.5)
    except TypeError:
        translated_small2.append(trans.text)
        time.sleep(0.5)
        continue

187it [05:18,  1.70s/it]


In [10]:
len(translated_small) + len(translated_small2)

768

In [12]:
translated=translated_small+translated_small2

In [8]:
data['google_small']=translated_small

In [16]:
data['google_small']=translated

In [18]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
data.to_csv('data/book_google.csv', index=False, encoding='utf-8-sig')

OSError: Cannot save file into a non-existent directory: 'data'

In [7]:
translated_big=[]
translator=googletrans.Translator()
for idx, val in tqdm(data.iterrows()):
    for_translate=val['big']
    try:
        trans=translator.translate(for_translate, src='ko', dest='en')
        translated_big.append(trans.text)
        time.sleep(0.5)
    except TypeError:
        translated_big.append(trans.text)
        time.sleep(0.5)
        continue

768it [21:37,  1.69s/it]


In [24]:
data2=data[184:]
translated_big2=[]
translator=googletrans.Translator()
for idx, val in tqdm(data2.iterrows()):
    for_translate=val['small']
    try:
        trans=translator.translate(for_translate, src='ko', dest='en')
        translated_big2.append(trans.text)
        time.sleep(0.5)
    except TypeError:
        translated_big2.append(trans.text)
        time.sleep(0.5)
        continue

584it [07:11,  1.35it/s]


In [26]:
translated=translated_big+translated_big2

In [8]:
data['google_big']=translated_big

In [9]:
data.to_csv('data/book_google3.csv', index=False, encoding='utf-8-sig')

OSError: Cannot save file into a non-existent directory: 'data'

In [41]:
data['kakao_small'].str.replace(' ', '"')

0      Santiago"was"an"old"fisherman"who"fished"alone...
1      For"the"first"forty"days"he"was"with"a"boy,"bu...
2      The"boy"was"heartbroken"by"the"sight"of"the"ol...
3      The"sails,"swathed"here"and"there"in"flakes"of...
4      Santiago"was"skinny"and"had"deep"wrinkles"on"t...
                             ...                        
763    Mrs."Cheng"asked"the"guide,""What"is"that?""At...
764    "This"is"Tiburon.""The"guide"replied,""It's"a"...
765    "⁇""Oh"my"god,"I"didn't"expect"sharks"to"have"...
766    "⁇""Really,"I've"never"seen"it"before.""⁇""The...
767    In"the"cabin"on"the"road,"the"old"man"was"fall...
Name: kakao_small, Length: 768, dtype: object

Cosine similarity using BERT transformer

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.']

sentences2 = ['He was an old man who fished alone with a small sailing boat in the Gulf of Mexico, and for eighty-four days he went out to sea and could not catch a fish.', #문학동네
              'He was an old man riding on a rock boat in the Gulf of Mexico and fishing alone, and he could not catch a fish four days after eighty days.', #민음사
              'He was an old man riding on a rock boat in the Gulf Current, fishing alone, and it was now 84 days when not a single fish could be caught.', #열린책들
              'He was an old man who fished alone in a small boat in an area of the Gulf of Mexico until today, 84 days without catching even one fish.', #시공사
              'He was an old man who was living alone on a rock boat in the Gulf of Mexico, but he had not caught any meat, and it had already been 84 days.', #다상
              'He was an old man who had laid out a rock boat in the Gulf of Mexico and was a lone fisherman, and the day was already 84 days without a fish.', #삼지사
              'He was an old man who was alone in a carton, going out to the Gulf of Mexico to catch fish, and he had not caught a single fish in the past 84 days.', #북로드
              'He was an old man who was alone in fishing in the waters of the Gulf of Mexico, and he could not catch even a small fish until the fourth day of the year.', #계몽사
              ]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \n{} \nScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man who fished alone with a small sailing boat in the Gulf of Mexico, and for eighty-four days he went out to sea and could not catch a fish. 
Score: 0.8891
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man riding on a rock boat in the Gulf of Mexico and fishing alone, and he could not catch a fish four days after eighty days. 
Score: 0.8511
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man riding on a rock boat in the Gulf Current, fishing alone, and it was now 84 days when not a single fish could be caught. 
Score: 0.8848
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old ma

In [ ]:
from collections import Counter
original=[
          'He','was','an','old','man','who','fished','alone','in','a','skiff','in','the', 'Gulf','Stream','and','he','had','gone','eighty-four','days','now','without','taking','a','fish'
]
translation=[
          'He','was','an','old','man','who','was','fishing','alone','in','a','small','boat','in','the','waters','of','the','Gulf','of','Mexico','and', 'he', 'could', 'not', 'catch', 'a', 'small', 'fish', 'until', 'the', 'eighth', 'and', 'fourth', 'days', 'passed' 

]
list((Counter(original)-Counter(translation)).elements())

['fished',
 'skiff',
 'Stream',
 'had',
 'gone',
 'eighty-four',
 'now',
 'without',
 'taking']